# AI LAB 11
### Name: Rasikh Ali
### Rollnumber: BSEM-F19-060

<div class="alert alert-block alert-info">
    Using <b>Python </b> v3.8.18
    <br>
    with <b>Jupyter </b> v7.4.9
</div>

# Libraries

In [57]:
import pandas as pd
import numpy as np
import pickle

from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data reading / loading

In [ ]:
df = pd.read_csv("data.csv")
df

In [ ]:
print(df)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

# Data Exploration

In [ ]:
# Data View
df.head()
df.head(2)

In [ ]:
df.tail()
df.tail(2)

In [ ]:
print(f"Number of rows {df.shape[0]}, Number of columns {df.shape[1]}")
print(df.shape)

In [ ]:
print("-- Attributes in Data --")
for cols in df.columns:
    print(cols)

In [ ]:
print("-- Number of instances in Data --")
print(df.count())

In [ ]:
df['city'].unique()

In [ ]:
print("-- Number of Unique Values in Data --")
print(df.nunique())

In [ ]:
print("-- Number of Null Values in Data --")
print(df.isnull().sum())

In [ ]:
print("-- Details of Data --")
df.describe()

In [ ]:
print("-- Insights of Data --")
df.info()

# Data Pre-Processing

### Manage Null-Values

In [ ]:
print("-- Number of Null Values in Data --")
print(df.isnull().sum())

In [ ]:
print("-- Number of Null Values in Data --")
print(df['street'].isnull().sum())

In [ ]:
# For Object/String Types
df['street']   = df['street'].fillna(df['street'].mode()[0])
df['city']     = df['city'].fillna(df['city'].mode()[0])
df['statezip'] = df['statezip'].fillna(df['statezip'].mode()[0])
df['country']  = df['country'].fillna(df['country'].mode()[0])

In [ ]:
def fillNaObjMode(col):
    for i in col:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['street', 'city', 'statezip', 'country']
fillNaObjMode(columns)

In [ ]:
def fillNaMean(col):
    for i in col:
        df[i] = df[i].fillna(df[i].mean())

columns = ['price','sqft_living','sqft_lot','sqft_above','sqft_basement']
fillNaMean(columns)

In [ ]:
def fillNaMode(col):
    for i in col:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['bedrooms','bathrooms','floors','waterfront','view','yr_built']
fillNaMode(columns)

In [ ]:
df.drop('date', axis=1, inplace=True)
df.drop('street', axis=1, inplace=True)

In [ ]:
df.head(2)

In [ ]:
print(df.isnull().sum())

In [ ]:
df['price'] = df['price'].astype('int64')
def changetoint64(col):
    for i in col:
        df[i] = df[i].astype('int64')
        
columns =[
    'price','bedrooms','bathrooms','sqft_living','sqft_lot',
    'floors','waterfront','view','sqft_above','sqft_basement',
    'yr_built']
changetoint64(columns)

In [ ]:
print("-- Insights of Data --")
df.info()

In [ ]:
def fillNaObjMode(col):
    for i in col:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['city', 'statezip', 'country']
fillNaObjMode(columns)

In [ ]:
df['city'].unique()

In [ ]:
df['statezip'].unique()

In [ ]:
df['country'].unique()

In [ ]:
df_encoded = df.copy()

In [ ]:
def encodeCols(cols):
    for i in cols:
        data = pd.DataFrame({i:df[i].unique()})
        data_label_encoder = LabelEncoder()
        data_label_encoder.fit(np.ravel(data))
        df_encoded[i] = data_label_encoder.transform(df[i]) 

columns = ['city','statezip','country']
encodeCols(columns)

In [ ]:
# Labels
country = pd.DataFrame({'country':df['country'].unique()})

# Initializing Label Encoders
country_label_encoder = LabelEncoder()

# Training Label Encoder
country_label_encoder.fit(np.ravel(country))

In [ ]:
df_encoded['country'] = country_label_encoder.transform(df['country']) 

In [ ]:
# All the Attributes are Numerical 
df_encoded.info()

In [ ]:
df_encoded.to_csv(r'encoded-data.csv', index = False, header = True)

# BREAK 3:10

# Train-Test Splitting

In [ ]:
traindata, testdata = train_test_split(df_encoded, test_size=0.2, shuffle=False)

In [ ]:
traindata.head(2)

In [ ]:
testdata.head(2)

In [ ]:
X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

In [ ]:
train_x = traindata.iloc[:, 1:]
train_x.head()

In [ ]:
train_y = traindata.iloc[:, 0]
train_y.head()

In [ ]:
test_x = testdata.iloc[:, 1:]
test_x.head()

In [ ]:
test_y = testdata.iloc[:, 0]
test_y.head()

# Training Model Using Support Vector Classifier

In [ ]:
print("-- Training using SVC on Training Data --")
print("-- Parameters & Values: ", end='')

model_svr = SVR(gamma='auto')
model_svr.fit(train_x, np.ravel(train_y))

print(model_svr)

In [ ]:
# Saving Trained Model
pickle.dump(model_svr, open('model_svc.pkl', 'wb'))

In [ ]:
# Load saved Model
model = pickle.load(open('model_svc.pkl', 'rb'))

In [ ]:
model_predictions = model.predict(test_x)

testdata_predict = testdata.copy(deep=True)
pd.options.mode.chained_assignment = None

testdata_predict['Prediction'] = model_predictions

In [ ]:
# Printing Testing Data
print("-- Testing Data with Prediction --")
# pd.set_option("display.max_rows", None, "display.max_columns", None)
testdata_predict

In [58]:
mse = mean_squared_error(testdata_predict['price'], testdata_predict['Prediction'])

print("-- mean squared error: ", end='')
print(mse)

-- mean squared error: 1063101319147.0159


In [59]:
rmse = root_mean_squared_error(testdata_predict['price'], testdata_predict['Prediction'])
print("-- root mean squared error: ", end='')
print(rmse)

-- root mean squared error: 1031068.0477771658


In [60]:
mae = mean_absolute_error(testdata_predict['price'], testdata_predict['Prediction'])
print("-- mean absolute error: ", end='')
print(mae)

-- mean absolute error: 291937.8900661286
